<a href="https://colab.research.google.com/github/gouthamans14/deeplearning-tesnorflow/blob/main/NLP/NLP_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Fundementals using Tensorflow

In [24]:
! nvidia-smi

Mon Jul  3 05:14:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [25]:
!wget https://raw.githubusercontent.com/gouthamans14/deeplearning-tesnorflow/main/extras/helper_functions.py

--2023-07-03 05:14:45--  https://raw.githubusercontent.com/gouthamans14/deeplearning-tesnorflow/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-07-03 05:14:45 (86.5 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [26]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

### Get a text dataset


In [27]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-07-03 05:14:45--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.126.128, 74.125.70.128, 74.125.132.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-07-03 05:14:45 (144 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



### Visualizing a text data

In [28]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [29]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [30]:
train_df[train_df["target"]==0]

,id,keyword,location,text,target
15,23,NaN,NaN,What's up man?,0
16,24,NaN,NaN,I love fruits,0
17,25,NaN,NaN,Summer is lovely,0
18,26,NaN,NaN,My car is so fast,0
19,28,NaN,NaN,What a goooooooaaaaaal!!!!!!,0
...,...,...,...,...,...
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0
7582,10834,wrecked,NaN,Cramer: Iger's 3 words that wrecked Disney's s...,0
7584,10837,NaN,NaN,These boxes are ready to explode! Exploding Ki...,0
7587,10841,NaN,NaN,Sirens everywhere!,0


In [31]:
train_df_shuffled = train_df.sample(random_state=42, frac=1) #frac - what percent of data in percentage to be shuffled 1 - 100%

In [32]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [33]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [34]:
100*(train_df.target.value_counts()/train_df.shape[0]) # Almost balanced

0    57.034021
1    42.965979
Name: target, dtype: float64

In [35]:
len(train_df), len(test_df)

(7613, 3263)

In [36]:
# Lets visualize some random samples

import random

random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"Target: {target}")
  print(f"Real disaster" if target>0 else "Not real disaster")
  print(f"Text: \n {text}\n")

Target: 1
Real disaster
Text: 
 Lightning strike in the distance via /r/pics http://t.co/iDmhSwewQw #pics

Target: 0
Not real disaster
Text: 
 @QuotesTTG Save the panicking for when you get to Helios. ;)

Target: 0
Not real disaster
Text: 
 Patience Jonathan On The Move To Hijack APC In BayelsaåÊState http://t.co/Vh8QtbyPZt

Target: 0
Not real disaster
Text: 
 Ali you flew planes and ran into burning buildings why are you making soup for that man child?! #BooRadleyVanCullen

Target: 0
Not real disaster
Text: 
 @ACOUSTICMALOLEY no he was blazing it



In [37]:
from sklearn.model_selection import train_test_split


In [38]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) #

In [39]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                           train_df_shuffled["target"].to_numpy(),
                                                                           test_size=0.1,
                                                                           random_state=42)

In [40]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

### Converting text to numbers
In NLP, there are two main concepts for turning text into numbers:

* Tokenization - A straight mapping from word or character or sub-word to a numerical value. There are three main levels of tokenization:
1) Using word-level tokenization with the sentence "I love TensorFlow" might result in "I" being 0, "love" being 1 and "TensorFlow" being 2. In this case, every word in a sequence considered a single token.<br>

2) Character-level* tokenization, such as converting the letters A-Z to values 1-26. In this case, every character in a sequence considered a single token.<br>

3) Sub-word tokenization is in between word-level and character-level tokenization. It involves breaking invidual words into smaller parts and then converting those smaller parts into numbers. For example, "my favourite food is pineapple pizza" might become "my, fav, avour, rite, fo, oo, od, is, pin, ine, app, le, piz, za". After doing this, these sub-words would then be mapped to a numerical value. In this case, every word could be considered multiple tokens.<br>

* *Embeddings* - An embedding is a representation of natural language which can be learned. Representation comes in the form of a feature vector. For example, the word "dance" could be represented by the 5-dimensional vector [-0.8547, 0.4559, -0.3332, 0.9877, 0.1112]. It's important to note here, the size of the feature vector is tuneable. There are two ways to use embeddings:<br>
<br>
1) Create your own embedding - Once your text has been turned into numbers (required for an embedding), you can put them through an embedding layer (such as tf.keras.layers.Embedding) and an embedding representation will be learned during model training.<br>
2) Reuse a pre-learned embedding - Many pre-trained embeddings exist online. These pre-trained embeddings have often been learned on large corpuses of text (such as all of Wikipedia) and thus have a good underlying representation of natural language. You can use a pre-trained embedding to initialize your model and fine-tune it to your own specific task.

### Text Vectorization

In [41]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [43]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [44]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [51]:
#Use the default TextVectorization Parameters
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                  split ="whitespace",
                                    ngrams=None,
                                    output_mode = "int",  #it can also be tf-idf
                                    output_sequence_length = None,
                                   # pad_to_max_tokens=True
                                    )

In [66]:
round(sum([len(i.split()) for i in train_sentences])/train_sentences.shape[0])

15

In [67]:
max_vocab_length = 10000
max_length =15

text_vectorizer = TextVectorization(max_tokens =max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length =max_length)

In [68]:
#Fit the text vecotrizer to the training data
text_vectorizer.adapt(train_sentences)

In [69]:
sample_sentence ="There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [73]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 @Jude_Mugabi not that all abortions get you traumatised. At times you are okay with the decision due to reasons like rape       

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1,  34,  16,  44,   1,  52,  12, 412,  17, 252,  12,  22, 734,
         14,   2]])>

In [83]:
# Get this unique wprds int the vocab

words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

In [84]:
len(words_in_vocab)

10000

In [85]:
top_5_words

['', '[UNK]', 'the', 'a', 'in']

In [86]:
bottom_5_words

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

### Creating an Embedding using an Embedding Layer

Parameters in an embedding layer:

* Input_dim = The size of our vocab
* Output_dim = The size of the outpur embedding vector for example a value  of100 mean each token gets represented by a 100 vector long
* Input Length = Length of the sequences passed to embedding layer
*

In [103]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim =128,
                             embeddings_initializer = "uniform",
                             input_length = max_length,
                             )

In [104]:
embedding

In [105]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original Text: \n {random_sentence}")
print("\n\nEmbedded Version: \n")
embedding(text_vectorizer([random_sentence]))

Original Text: 
 Georgia sinkhole closes road swallows whole pond http://t.co/cPEQv52LNA


Embedded Version: 



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0344486 ,  0.02119583, -0.04749752, ..., -0.02439073,
          0.00130534, -0.03613932],
        [ 0.02449404,  0.00429201,  0.01115889, ..., -0.01937102,
          0.03189577,  0.04164121],
        [-0.02198155, -0.03198766, -0.00400855, ...,  0.04744294,
         -0.01062214,  0.00848787],
        ...,
        [-0.02834278,  0.00750755, -0.04900932, ..., -0.03578069,
          0.0486724 ,  0.01193001],
        [-0.02834278,  0.00750755, -0.04900932, ..., -0.03578069,
          0.0486724 ,  0.01193001],
        [-0.02834278,  0.00750755, -0.04900932, ..., -0.03578069,
          0.0486724 ,  0.01193001]]], dtype=float32)>

### Mode 0: Baseline Model

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [111]:
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB())
])

# Fit the pipeline to the trainig data

model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [113]:
baseline_score = model_0.score(val_sentences,val_labels)

In [115]:
baseline_score*100

79.26509186351706

In [118]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [116]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [119]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A Simple Dense Model

In [120]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"


In [124]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype =tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs, name="Model_1_dense")


In [125]:
model_1.summary()

Model: "Model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 dense (Dense)               (None, 15, 1)             129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_1.compile(loss=tf.losses.BinaryCrossentropy(),
                optimizer = tf.optimizers.Adam(),
                metrics=["accuracy"])

model_1.fit(x =train_sentences, y = train_labels,
            epochs=5,
            validation_data =(val_sentences,val_labels),
            callbacks =[create_tensorboard_callback(SAVE_DIR,
                                                    "model_1_dense")])

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [141]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [ ]:
model_1.compile(loss=tf.losses.BinaryCrossentropy(),
                optimizer = tf.optimizers.Adam(),
                metrics=["accuracy"])

model_1.fit(x =train_sentences, y = train_labels,
            epochs=5,
            validation_data =(val_sentences,val_labels),
            callbacks =[create_tensorboard_callback(SAVE_DIR,
                                                    "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230703-073152
Epoch 1/5
215/215 [==============================] - 29s 102ms/step - loss: 0.6153 - accuracy: 0.6935 - val_loss: 0.5378 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4431 - accuracy: 0.8183 - val_loss: 0.4737 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3485 - accuracy: 0.8606 - val_loss: 0.4566 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2854 - accuracy: 0.8892 - val_loss: 0.4629 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2390 - accuracy: 0.9117 - val_loss: 0.4792 - val_accuracy: 0.7887


In [145]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 7ms/step - loss: 0.4792 - accuracy: 0.7887


[0.4791821539402008, 0.7887139320373535]